# Mapping political data for the USA

This Python notebook was written to render politically-related data onto maps of the USA.

We will produce maps that show:

1. A shift in support for trump from a a recent poll as compared to the support he had in the Presidential Elections of 2024  
2. The percentage of college graduates in each state which correlates with support trump had in the Presidential Elections of 2024  
3. The dependency on Federal Transfers which also correlates with support trump had in the Presidential Elections of 2024  

We will use publicly available data to produce these maps.

Once generated, the dynamic maps are exported to HTML format, which when previewed in a browser, also allow a static image of the map to be saved in PNG format.

## Shifting support for trump

This data was downloaded from [https://www.newsweek.com/map-shows-donald-trumps-approval-rating-in-each-state-after-nine-months-10894967](https://www.newsweek.com/map-shows-donald-trumps-approval-rating-in-each-state-after-nine-months-10894967). Specifically, after opening the page, I used the F12 function key to open a debugger. In the debugger showing the contents of the HtML file, search for the JavaScript variable named "_Flourish_data" then copy this value and then use a JSON formatter (for better readability) to produce the json_string we use in the code below.

## Educational attainment by state

Data was downloaded from [https://fred.stlouisfed.org/release/tables?eid=391444&rid=330](https://fred.stlouisfed.org/release/tables?eid=391444&rid=330) - you must have a FRED account to download data.

Specifically, we select all states in the list (excluding District of Columbia) and then clicked on the "Add Data to List" button.  
On the next screen, we accept the default selection "Levels" in the "Units" column for each state, and then choose all the dates in the column "Vintage Dates When Data Were Revised" and click on the "Add Series to My Data List" button.  
On the next screen, simply click on the "Download Data" button.  
On this final screen, select a date range for observations, choose the "Zipped CSV" format, and click the "Download Data" button.  
This project already contains such a downloaded file [Educational_Attainment_by_State_2024.zip](Educational_Attainment_by_State_2024.zip) which when exploded, creates the files:
- [data/annual_1.csv](data/annual_1.csv)
- [data/annual_2.csv](data/annual_2.csv)
- [data/README.txt](data/README.txt)

## Federal Transfers dependency

Data was downloaded from [https://rockinst.org/issue-areas/fiscal-analysis/balance-of-payments-portal/](https://rockinst.org/issue-areas/fiscal-analysis/balance-of-payments-portal/) - click on the blue button in the center of the screen labelled "Per Capita Expenditures & Receipts" to see each state with or without COVID assistance. Each state in the Union requires a separate download. I specifically chose to download data that excluded COVID payments by the Federal Government to households. As a result, 50 seprate files were downloaded also included in this project named "data/ReceiptsExpendituresTable_*STATE*.csv" where *STATE* is the name of one of the 50 states in the Union.

First, let's import the libraries we will use for this exercise

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from shapely.geometry import Polygon
from io import StringIO
import os
import requests
from glob import glob 

In [2]:
# Global variables and options

pd.set_option('display.max_rows', 50) # Increase the default from 10

In [3]:
# Let's ensure the output folder exists
output_folder = "output"
os.makedirs(output_folder, exist_ok=True)

For mapping annotations, we need to calculate the centroid of each state in order to place the text annotation near the weighted center of each state. Luckily, we can find such data in the public domain too at [https://raw.githubusercontent.com/PublicaMundi/MappingAPI/master/data/geojson/us-states.json](https://raw.githubusercontent.com/PublicaMundi/MappingAPI/master/data/geojson/us-states.json) however, we will read that URL programmatically using the requests library.

In [4]:
# We will use this in each of the maps we draw so let's do it once at the top of the file.

# Note: Plotly Express often handles state name matching internally, but using GeoJSON
# is the robust way to ensure a match with a specific key.
URL_STATES = "https://raw.githubusercontent.com/PublicaMundi/MappingAPI/master/data/geojson/us-states.json"
states_geojson = requests.get(URL_STATES).json()

In [5]:
# For leader lines for small states in the northeast and mid-atlantic regions - used in all maps

dict_state_leaderlines = [
      {"name": "Massachussettes", "leadlat": 42.467688, "leadlon": -70.238972}
    , {"name": "Rhode Island", "leadlat": 40.991884, "leadlon": -70.156546}
    , {"name": "Connecticut", "leadlat": 40.921887, "leadlon": -71.581369}
    , {"name": "New Hampshire", "leadlat": 42.915917, "leadlon": -70.271845}
    , {"name": "Vermont", "leadlat": 45.650633, "leadlon": -72.693992}
    , {"name": "Delaware", "leadlat": 38.679105, "leadlon": -74.487365}
    , {"name": "New Jersey", "leadlat": 39.720863, "leadlon": -73.734956}
]

df_states_leaderlines = pd.DataFrame(dict_state_leaderlines)
df_states_leaderlines

,name,leadlat,leadlon
0,Massachussettes,42.467688,-70.238972
1,Rhode Island,40.991884,-70.156546
2,Connecticut,40.921887,-71.581369
3,New Hampshire,42.915917,-70.271845
4,Vermont,45.650633,-72.693992
5,Delaware,38.679105,-74.487365
6,New Jersey,39.720863,-73.734956


In [6]:
# Calculate centroids for each state to place the text on the map
dict_state_centroids = {}
for s in range(len(states_geojson["features"])):
    state_name = states_geojson["features"][s]["properties"]["name"]
    state_coords = states_geojson["features"][s]["geometry"]["coordinates"]
    #state_centroid = Polygon(contig_coords).centroid
    #dict_state_centroids[state_name] = {"lat": round(state_centroid.y, 4), "lon": round(state_centroid.x, 4)}
    total_area = 0.0
    sum_of_weighted_lon = 0.0
    sum_of_weighted_lat = 0.0
    for contig_coords in state_coords:
        if len(contig_coords) == 1:
            contig_coords = contig_coords[0]
        polygon = Polygon(contig_coords)
        contig_area = polygon.area
        contig_cent = polygon.centroid
        contig_lat = contig_cent.y
        contig_lon = contig_cent.x

        sum_of_weighted_lat += contig_area * contig_lat
        sum_of_weighted_lon += contig_area * contig_lon
        total_area += contig_area
    
    if contig_area > 0:
        dict_state_centroids[state_name] = {"lat": round(sum_of_weighted_lat / total_area, 4), "lon": round(sum_of_weighted_lon / total_area, 4)}
        
df_state_centroids = pd.DataFrame(dict_state_centroids).T
df_state_centroids = df_state_centroids.reset_index().rename(columns={"index": "name"})
df_state_centroids = pd.merge(df_state_centroids, df_states_leaderlines, on="name", how="left")
# Fill the NaNs in leadlat and leadlon resulting from the left join with values from lat and lon
df_state_centroids["leadlat"] = df_state_centroids["leadlat"].fillna(df_state_centroids["lat"])
df_state_centroids["leadlon"] = df_state_centroids["leadlon"].fillna(df_state_centroids["lon"])
display(df_state_centroids)

,name,lat,lon,leadlat,leadlon
0,Alabama,32.7899,-86.8278,32.7899,-86.8278
1,Alaska,64.2204,-152.5427,64.2204,-152.5427
2,Arizona,34.2934,-111.6633,34.2934,-111.6633
3,Arkansas,34.8982,-92.4409,34.8982,-92.4409
4,California,37.2539,-119.6144,37.2539,-119.6144
...,...,...,...,...,...
47,Washington,47.3754,-120.4490,47.3754,-120.4490
48,West Virginia,38.6384,-80.6167,38.6384,-80.6167
49,Wisconsin,44.6350,-90.0142,44.6350,-90.0142
50,Wyoming,43.0007,-107.5519,43.0007,-107.5519


In [7]:
df_states_leaderlines = pd.merge(df_states_leaderlines, df_state_centroids[["name", "lat", "lon"]], on="name", how="inner")

## Shifting support for trump

In [8]:
str_json_state_approval = '''[
      {"id": 1, "metadata": [17, 55, "38"], "name": "Alabama", "value": [17]}
    , {"id": 2, "metadata": [3, 49, "46"], "name": "Alaska", "value": [3]}
    , {"id": 4, "metadata": [-10, 43, "53"], "name": "Arizona", "value": [-10]}
    , {"id": 5, "metadata": [17, 56, "39"], "name": "Arkansas", "value": [17]}
    , {"id": 6, "metadata": [-38, 29, "67"], "name": "California", "value": [-38]}
    , {"id": 8, "metadata": [-21, 37, "58"], "name": "Colorado", "value": [-21]}
    , {"id": 9, "metadata": [-28, 34, "62"], "name": "Connecticut", "value": [-28]}
    , {"id": 10, "metadata": [-27, 35, "62"], "name": "Delaware", "value": [-27]}
    , {"id": 12, "metadata": [-4, 46, "50"], "name": "Florida", "value": [-4]}
    , {"id": 13, "metadata": [-7, 44, "51"], "name": "Georgia", "value": [-7]}
    , {"id": 15, "metadata": [-54, 21, "75"], "name": "Hawaii", "value": [-54]}
    , {"id": 16, "metadata": [25, 60, "35"], "name": "Idaho", "value": [25]}
    , {"id": 17, "metadata": [-29, 34, "63"], "name": "Illinois", "value": [-29]}
    , {"id": 18, "metadata": [10, 53, "43"], "name": "Indiana", "value": [10]}
    , {"id": 19, "metadata": [-6, 45, "51"], "name": "Iowa", "value": [-6]}
    , {"id": 20, "metadata": [10, 53, "43"], "name": "Kansas", "value": [10]}
    , {"id": 21, "metadata": [14, 54, "40"], "name": "Kentucky", "value": [14]}
    , {"id": 22, "metadata": [5, 50, "45"], "name": "Louisiana", "value": [5]}
    , {"id": 23, "metadata": [-19, 38, "57"], "name": "Maine", "value": [-19]}
    , {"id": 24, "metadata": [-42, 27, "69"], "name": "Maryland", "value": [-42]}
    , {"id": 25, "metadata": [-40, 28, "68"], "name": "Massachusetts", "value": [-40]}
    , {"id": 26, "metadata": [-13, 41, "54"], "name": "Michigan", "value": [-13]}
    , {"id": 27, "metadata": [-23, 36, "59"], "name": "Minnesota", "value": [-23]}
    , {"id": 28, "metadata": [11, 53, "42"], "name": "Mississippi", "value": [11]}
    , {"id": 29, "metadata": [3, 49, "46"], "name": "Missouri", "value": [3]}
    , {"id": 30, "metadata": [20, 58, "38"], "name": "Montana", "value": [20]}
    , {"id": 31, "metadata": [15, 55, "40"], "name": "Nebraska", "value": [15]}
    , {"id": 32, "metadata": [-8, 44, "52"], "name": "Nevada", "value": [-8]}
    , {"id": 33, "metadata": [-22, 37, "59"], "name": "New Hampshire", "value": [-22]}
    , {"id": 34, "metadata": [-25, 36, "61"], "name": "New Jersey", "value": [-25]}
    , {"id": 35, "metadata": [-22, 37, "59"], "name": "New Mexico", "value": [-22]}
    , {"id": 36, "metadata": [-32, 32, "64"], "name": "New York", "value": [-32]}
    , {"id": 37, "metadata": [-10, 43, "53"], "name": "North Carolina", "value": [-10]}
    , {"id": 38, "metadata": [17, 57, "40"], "name": "North Dakota", "value": [17]}
    , {"id": 39, "metadata": [-5, 45, "50"], "name": "Ohio", "value": [-5]}
    , {"id": 40, "metadata": [15, 55, "40"], "name": "Oklahoma", "value": [15]}
    , {"id": 41, "metadata": [-36, 30, "66"], "name": "Oregon", "value": [-36]}
    , {"id": 42, "metadata": [-9, 43, "52"], "name": "Pennsylvania", "value": [-9]}
    , {"id": 44, "metadata": [-38, 29, "67"], "name": "Rhode Island", "value": [-38]}
    , {"id": 45, "metadata": [4, 50, "46"], "name": "South Carolina", "value": [4]}
    , {"id": 46, "metadata": [17, 56, "39"], "name": "South Dakota", "value": [17]}
    , {"id": 47, "metadata": [11, 53, "42"], "name": "Tennessee", "value": [11]}
    , {"id": 48, "metadata": [-3, 47, "50"], "name": "Texas", "value": [-3]}
    , {"id": 49, "metadata": [14, 54, "40"], "name": "Utah", "value": [14]}
    , {"id": 50, "metadata": [-48, 24, "72"], "name": "Vermont", "value": [-48]}
    , {"id": 51, "metadata": [-22, 37, "59"], "name": "Virginia", "value": [-22]}
    , {"id": 53, "metadata": [-35, 30, "65"], "name": "Washington", "value": [-35]}
    , {"id": 54, "metadata": [28, 61, "33"], "name": "West Virginia", "value": [28]}
    , {"id": 55, "metadata": [-11, 43, "54"], "name": "Wisconsin", "value": [-11]}
    , {"id": 56, "metadata": [34, 65, "31"], "name": "Wyoming", "value": [34]}
]'''

# Load the JSON string into a pandas DataFrame

df_state_approval = pd.read_json(StringIO(str_json_state_approval))
display(df_state_approval)

,id,metadata,name,value
0,1,"[17, 55, 38]",Alabama,[17]
1,2,"[3, 49, 46]",Alaska,[3]
2,4,"[-10, 43, 53]",Arizona,[-10]
3,5,"[17, 56, 39]",Arkansas,[17]
4,6,"[-38, 29, 67]",California,[-38]
5,8,"[-21, 37, 58]",Colorado,[-21]
6,9,"[-28, 34, 62]",Connecticut,[-28]
7,10,"[-27, 35, 62]",Delaware,[-27]
8,12,"[-4, 46, 50]",Florida,[-4]
9,13,"[-7, 44, 51]",Georgia,[-7]


In [9]:
# Extract the three elements from the 'metadata' list into new columns
# We use the .str[index] accessor to safely extract elements from the list/array in the column.
df_state_approval["Difference"] = df_state_approval["metadata"].str[0].astype(float)
df_state_approval["Approval"] = df_state_approval["metadata"].str[1].astype(float)
df_state_approval["Disapproval"] = df_state_approval["metadata"].str[2].astype(float)

# 3. Select only the requested columns ('name', 'difference', 'approval', 'disapproval')
final_df_state_approval = df_state_approval[["name", "Difference", "Approval", "Disapproval"]]

print("\nDataFrame Shape:", final_df_state_approval.shape)

display(final_df_state_approval)


DataFrame Shape: (50, 4)


,name,Difference,Approval,Disapproval
0,Alabama,17.0,55.0,38.0
1,Alaska,3.0,49.0,46.0
2,Arizona,-10.0,43.0,53.0
3,Arkansas,17.0,56.0,39.0
4,California,-38.0,29.0,67.0
5,Colorado,-21.0,37.0,58.0
6,Connecticut,-28.0,34.0,62.0
7,Delaware,-27.0,35.0,62.0
8,Florida,-4.0,46.0,50.0
9,Georgia,-7.0,44.0,51.0


In [10]:
df_merged_state_approval = pd.merge(final_df_state_approval, df_state_centroids, on="name", how="inner")
# Create the label text for the map: State Name (Difference)
df_merged_state_approval["label_text"] = df_merged_state_approval.apply(
    lambda row: f"{row["name"]}<br>({row["Difference"]:.0f})", axis=1
)
display(df_merged_state_approval)

,name,Difference,Approval,Disapproval,lat,lon,leadlat,leadlon,label_text
0,Alabama,17.0,55.0,38.0,32.7899,-86.8278,32.789900,-86.827800,Alabama<br>(17)
1,Alaska,3.0,49.0,46.0,64.2204,-152.5427,64.220400,-152.542700,Alaska<br>(3)
2,Arizona,-10.0,43.0,53.0,34.2934,-111.6633,34.293400,-111.663300,Arizona<br>(-10)
3,Arkansas,17.0,56.0,39.0,34.8982,-92.4409,34.898200,-92.440900,Arkansas<br>(17)
4,California,-38.0,29.0,67.0,37.2539,-119.6144,37.253900,-119.614400,California<br>(-38)
5,Colorado,-21.0,37.0,58.0,38.9993,-105.5488,38.999300,-105.548800,Colorado<br>(-21)
6,Connecticut,-28.0,34.0,62.0,41.6209,-72.7279,40.921887,-71.581369,Connecticut<br>(-28)
7,Delaware,-27.0,35.0,62.0,38.9827,-75.4976,38.679105,-74.487365,Delaware<br>(-27)
8,Florida,-4.0,46.0,50.0,28.6589,-82.5040,28.658900,-82.504000,Florida<br>(-4)
9,Georgia,-7.0,44.0,51.0,32.6480,-83.4465,32.648000,-83.446500,Georgia<br>(-7)


In [11]:
# 4. Create the Choropleth Map using Plotly Express
fig_state_approval = px.choropleth(
    df_merged_state_approval,
    # Match the 'name' column in the DataFrame with the 'name' property in the GeoJSON features
    geojson=states_geojson,
    locations="name",
    #locationmode="USA-states",
    featureidkey="properties.name",
    
    # Color based on the 'difference' column
    color="Difference",
    
    # Specify the geographic scope (USA)
    scope="usa",
    
    # Set the custom color scale: Red for high positive, Blue for high negative
    # The 'rdbu' (Red-Blue) colorscale is a good diverging option.
    color_continuous_scale="RdBu_r",
    
    # Ensure the color scale centers around zero for accurate representation of positive/negative difference
    color_continuous_midpoint=0,
    
    # Add all relevant data to the hover box
    hover_name="name",
    hover_data={
        "Approval": True,
        "Disapproval": True,
        "Difference": True,
        "name": False, # Hide duplicate name
        #'id': False,    # Hide id column
        #'metadata': False # Hide original metadata list
    },
    title="State Approval Difference (Approval - Disapproval)"
)

In [12]:
# Add Permanent Text Labels using Centroids
# We use go.Scattergeo to plot the text at the centroid coordinates
bogus = fig_state_approval.add_trace(
    go.Scattergeo(
        locationmode = "USA-states",
        lon = df_merged_state_approval["leadlon"],
        lat = df_merged_state_approval["leadlat"],
        text = df_merged_state_approval["label_text"],
        mode = "text",
        textfont = dict(
            size=12,  # Smaller font to reduce overlap
            color="black"
        ),
        showlegend = False,
        hoverinfo="skip"
    )
)

# Also add leader lines for those smaller states (northeast)

for _, row in df_states_leaderlines.iterrows():
    fig_state_approval.add_trace(go.Scattergeo(
        lon=[row["leadlon"], row["lon"]],
        lat=[row["leadlat"], row["lat"]],
        mode="lines",
        line=dict(color="green", width=1),
        showlegend=False,
        hoverinfo="skip"
    ))

In [13]:
# 7. Final layout adjustments
fig_state_approval.update_geos(
    visible=False,
    # Customize the map projection
    projection={"type": "albers usa"},
    #fitbounds="locations"
)

fig_state_approval.update_layout(
    margin={"r":0,"t":40,"l":0,"b":0},
    coloraxis_colorbar=dict(
        title="Approval Difference"
    ),
    geo={"scope": "usa"},
    title="State Approval Difference (Blue=Negative, Red=Positive)"
)

fig_state_approval.show()

In [14]:
fig_state_approval.write_html(f"{output_folder}/TrumpStateApprovalDifference.html")

## Educational Attainment

In [15]:
cols1 = [
     "observation_date"
    ,"GCT1502AK_20250911"
    ,"GCT1502AL_20250911"
    ,"GCT1502AR_20250911"
    ,"GCT1502AZ_20250911"
    ,"GCT1502CA_20250911"
    ,"GCT1502CO_20250911"
    ,"GCT1502CT_20250911"
    ,"GCT1502DE_20250911"
    ,"GCT1502FL_20250911"
    ,"GCT1502GA_20250911"
    ,"GCT1502HI_20250911"
    ,"GCT1502IA_20250911"
    ,"GCT1502ID_20250911"
    ,"GCT1502IL_20250911"
    ,"GCT1502IN_20250911"
    ,"GCT1502KS_20250911"
    ,"GCT1502KY_20250911"
    ,"GCT1502LA_20250911"
    ,"GCT1502MA_20250911"
    ,"GCT1502MD_20250911"
    ,"GCT1502ME_20250911"
    ,"GCT1502MI_20250911"
    ,"GCT1502MN_20250911"
    ,"GCT1502MO_20250911"
    ,"GCT1502MS_20250911"
    ,"GCT1502MT_20250911"
    ,"GCT1502NC_20250911"
    ,"GCT1502ND_20250911"
    ,"GCT1502NE_20250911"
    ,"GCT1502NH_20250911"
    ,"GCT1502NJ_20250911"
    ,"GCT1502NM_20250911"
    ,"GCT1502NV_20250911"
    ,"GCT1502NY_20250911"
    ,"GCT1502OH_20250911"
    ,"GCT1502OK_20250911"
    ,"GCT1502OR_20250911"
    ,"GCT1502PA_20250911"
]

renamed1 = {
     "observation_date": "date"
    ,"GCT1502AK_20250911": "Alaska"
    ,"GCT1502AL_20250911": "Alabama"
    ,"GCT1502AR_20250911": "Arkansas"
    ,"GCT1502AZ_20250911": "Arizona"
    ,"GCT1502CA_20250911": "California"
    ,"GCT1502CO_20250911": "Colorado"
    ,"GCT1502CT_20250911": "Connecticut"
    ,"GCT1502DE_20250911": "Delaware"
    ,"GCT1502FL_20250911": "Florida"
    ,"GCT1502GA_20250911": "Georgia"
    ,"GCT1502HI_20250911": "Hawaii"
    ,"GCT1502IA_20250911": "Iowa"
    ,"GCT1502ID_20250911": "Idaho"
    ,"GCT1502IL_20250911": "Illinois"
    ,"GCT1502IN_20250911": "Indiana"
    ,"GCT1502KS_20250911": "Kansas"
    ,"GCT1502KY_20250911": "Kentucky"
    ,"GCT1502LA_20250911": "Louisiana"
    ,"GCT1502MA_20250911": "Massachusetts"
    ,"GCT1502MD_20250911": "Maryland"
    ,"GCT1502ME_20250911": "Maine"
    ,"GCT1502MI_20250911": "Michigan"
    ,"GCT1502MN_20250911": "Minnesota"
    ,"GCT1502MO_20250911": "Missouri"
    ,"GCT1502MS_20250911": "Mississippi"
    ,"GCT1502MT_20250911": "Montana"
    ,"GCT1502NC_20250911": "North Carolina"
    ,"GCT1502ND_20250911": "North Dakota"
    ,"GCT1502NE_20250911": "Nebraska"
    ,"GCT1502NH_20250911": "New Hampshire"
    ,"GCT1502NJ_20250911": "New Jersey"
    ,"GCT1502NM_20250911": "New Mexico"
    ,"GCT1502NV_20250911": "Nevada"
    ,"GCT1502NY_20250911": "New York"
    ,"GCT1502OH_20250911": "Ohio"
    ,"GCT1502OK_20250911": "Oklahoma"
    ,"GCT1502OR_20250911": "Oregon"
    ,"GCT1502PA_20250911": "Pennsylvania"
}
df_education1 = pd.read_csv("data/annual_1.csv", usecols=cols1)
df_education1.rename(columns=renamed1, inplace=True)
# Convert date stamp in column date to year only
df_education1["date"] = "Year_" + (pd.to_datetime(df_education1["date"]).dt.year).astype(str)
display(df_education1)

,date,Alaska,Alabama,Arkansas,Arizona,California,Colorado,Connecticut,Delaware,Florida,...,Nebraska,New Hampshire,New Jersey,New Mexico,Nevada,New York,Ohio,Oklahoma,Oregon,Pennsylvania
0,Year_2006,26.9,21.1,18.2,25.5,29.0,34.3,33.7,27.0,25.3,...,26.9,31.9,33.4,25.3,20.8,31.2,23.0,22.1,27.5,25.4
1,Year_2007,26.0,21.4,19.3,25.3,29.5,35.0,34.7,26.1,25.8,...,27.5,32.5,33.9,24.8,21.8,31.7,24.1,22.8,28.3,25.8
2,Year_2008,27.3,22.0,18.8,25.1,29.6,35.6,35.6,27.5,25.8,...,27.1,33.3,34.4,24.7,21.9,31.9,24.1,22.2,28.1,26.3
3,Year_2009,26.6,22.0,18.9,25.6,29.9,35.9,35.6,28.7,25.3,...,27.4,32.0,34.5,25.3,21.8,32.4,24.1,22.7,29.2,26.4
4,Year_2010,27.9,21.9,19.5,25.9,30.1,36.4,35.5,27.8,25.8,...,28.6,32.8,35.4,25.0,21.7,32.5,24.6,22.9,28.8,27.1
5,Year_2011,26.4,22.3,20.3,26.6,30.3,36.7,36.2,28.8,25.8,...,27.9,33.4,35.3,25.6,22.5,32.9,24.7,23.8,29.3,27.0
6,Year_2012,28.0,23.3,21.0,27.3,30.9,37.5,37.1,29.5,26.8,...,29.0,34.6,36.2,26.1,22.4,33.4,25.2,23.8,29.9,27.8
7,Year_2013,28.0,23.5,20.6,27.4,31.0,37.8,37.2,29.8,27.2,...,29.4,34.6,36.6,26.4,22.5,34.1,26.1,23.8,30.7,28.7
8,Year_2014,28.0,23.5,21.4,27.6,31.7,38.3,38.0,30.6,27.3,...,29.5,35.0,37.4,26.4,23.1,34.5,26.6,24.2,30.8,29.0
9,Year_2015,29.7,24.2,21.8,27.7,32.3,39.2,38.3,30.9,28.4,...,30.2,35.7,37.6,26.5,23.6,35.0,26.8,24.6,32.2,29.7


In [16]:
cols2 = [
     "observation_date"
    ,"GCT1502RI_20250911"
    ,"GCT1502SC_20250911"
    ,"GCT1502SD_20250911"
    ,"GCT1502TN_20250911"
    ,"GCT1502TX_20250911"
    ,"GCT1502UT_20250911"
    ,"GCT1502VA_20250911"
    ,"GCT1502VT_20250911"
    ,"GCT1502WA_20250911"
    ,"GCT1502WI_20250911"
    ,"GCT1502WV_20250911"
    ,"GCT1502WY_20250911"
]
renamed2 = {
     "observation_date": "date"
    ,"GCT1502RI_20250911": "Rhode Island"
    ,"GCT1502SC_20250911": "South Carolina"
    ,"GCT1502SD_20250911": "South Dakota"
    ,"GCT1502TN_20250911": "Tennessee"
    ,"GCT1502TX_20250911": "Texas"
    ,"GCT1502UT_20250911": "Utah"
    ,"GCT1502VA_20250911": "Virginia"
    ,"GCT1502VT_20250911": "Vermont"
    ,"GCT1502WA_20250911": "Washington"
    ,"GCT1502WI_20250911": "Wisconsin"
    ,"GCT1502WV_20250911": "West Virginia"
    ,"GCT1502WY_20250911": "Wyoming"
}
df_education2 = pd.read_csv("data/annual_2.csv", usecols=cols2)
df_education2.rename(columns=renamed2, inplace=True)
# Convert date stamp in column date to year only
df_education2["date"] = "Year_" + (pd.to_datetime(df_education2["date"]).dt.year).astype(str)
display(df_education2)

,date,Rhode Island,South Carolina,South Dakota,Tennessee,Texas,Utah,Virginia,Vermont,Washington,Wisconsin,West Virginia,Wyoming
0,Year_2006,29.6,22.7,24.8,21.7,24.7,28.6,32.7,32.4,30.5,25.1,16.5,22.7
1,Year_2007,29.8,23.5,25.0,21.8,25.2,28.7,33.6,33.6,30.3,25.4,17.3,23.4
2,Year_2008,30.0,23.7,25.1,22.9,25.3,29.1,33.7,32.1,30.7,25.7,17.1,23.6
3,Year_2009,30.5,24.3,25.1,23.0,25.5,28.5,34.0,33.1,31.0,25.7,17.3,23.8
4,Year_2010,30.2,24.5,26.3,23.1,25.9,29.3,34.2,33.6,31.1,26.3,17.5,24.1
5,Year_2011,31.1,24.1,26.3,23.6,26.4,29.7,35.1,35.4,31.9,26.5,18.5,24.7
6,Year_2012,31.4,25.1,26.3,24.3,26.7,30.7,35.5,35.8,31.7,27.1,18.6,24.7
7,Year_2013,32.4,26.1,26.6,24.8,27.5,31.3,36.1,35.7,32.7,27.7,18.9,26.6
8,Year_2014,30.4,26.3,27.8,25.3,27.8,31.1,36.7,34.9,33.1,28.4,19.2,26.6
9,Year_2015,32.7,26.8,27.5,25.7,28.4,31.8,37.0,36.9,34.2,28.4,19.6,26.2


In [17]:
df_education_attain = pd.merge(df_education1, df_education2, on="date").set_index("date").sort_index().T
display(df_education_attain)

date,Year_2006,Year_2007,Year_2008,Year_2009,Year_2010,Year_2011,Year_2012,Year_2013,Year_2014,Year_2015,Year_2016,Year_2017,Year_2018,Year_2019,Year_2020,Year_2021,Year_2022,Year_2023,Year_2024
Alaska,26.9,26.0,27.3,26.6,27.9,26.4,28.0,28.0,28.0,29.7,29.6,28.8,30.2,30.2,31.9,32.8,30.6,32.2,32.8
Alabama,21.1,21.4,22.0,22.0,21.9,22.3,23.3,23.5,23.5,24.2,24.7,25.5,25.5,26.3,27.8,27.4,28.8,28.9,29.9
Arkansas,18.2,19.3,18.8,18.9,19.5,20.3,21.0,20.6,21.4,21.8,22.4,23.4,23.3,23.3,24.9,25.3,25.4,26.2,27.1
Arizona,25.5,25.3,25.1,25.6,25.9,26.6,27.3,27.4,27.6,27.7,28.9,29.4,29.7,30.2,33.0,32.4,33.0,33.5,34.7
California,29.0,29.5,29.6,29.9,30.1,30.3,30.9,31.0,31.7,32.3,32.9,33.6,34.2,35.0,36.9,36.2,37.0,37.5,38.1
Colorado,34.3,35.0,35.6,35.9,36.4,36.7,37.5,37.8,38.3,39.2,39.9,41.2,41.7,42.7,44.2,44.4,45.9,46.4,47.8
Connecticut,33.7,34.7,35.6,35.6,35.5,36.2,37.1,37.2,38.0,38.3,38.6,38.7,39.6,39.8,42.4,42.1,41.9,42.9,42.6
Delaware,27.0,26.1,27.5,28.7,27.8,28.8,29.5,29.8,30.6,30.9,31.0,31.5,31.3,33.2,34.7,35.6,36.5,36.5,36.0
Florida,25.3,25.8,25.8,25.3,25.8,25.8,26.8,27.2,27.3,28.4,28.6,29.7,30.4,30.7,33.7,33.2,34.3,34.9,35.8
Georgia,26.6,27.1,27.5,27.5,27.3,27.6,28.2,28.3,29.1,29.9,30.5,30.9,31.9,32.5,34.8,34.6,34.7,35.4,36.3


In [18]:
df_education_attain_2024 = df_education_attain["Year_2024"]
df_education_attain_2024 = df_education_attain_2024.reset_index().rename(columns={"index": "name"})
display(df_education_attain_2024)

,name,Year_2024
0,Alaska,32.8
1,Alabama,29.9
2,Arkansas,27.1
3,Arizona,34.7
4,California,38.1
5,Colorado,47.8
6,Connecticut,42.6
7,Delaware,36.0
8,Florida,35.8
9,Georgia,36.3


In [19]:
# 4. Create the Choropleth Map using Plotly Express
fig_education_attain = px.choropleth(
    df_education_attain_2024,
    # Match the 'name' column in the DataFrame with the 'name' property in the GeoJSON features
    geojson=states_geojson,
    locations="name",
    #locationmode="USA-states",
    featureidkey="properties.name",
    
    # Color based on the 'difference' column
    color="Year_2024",
    
    # Specify the geographic scope (USA)
    scope="usa",
    
    # Set the custom color scale: Red for high positive, Blue for high negative
    # The 'rdbu' (Red-Blue) colorscale is a good diverging option.
    color_continuous_scale="RdBu",
    
    # Ensure the color scale centers around zero for accurate representation of positive/negative difference
    color_continuous_midpoint=0,

    range_color=(df_education_attain_2024["Year_2024"].min(), df_education_attain_2024["Year_2024"].max()),
    
    # Add all relevant data to the hover box
    #hover_name="name",
    #hover_data={
    #    "Year_2024": True,
    #    "name": False, # Hide duplicate name
    #},
    custom_data=[
          df_education_attain_2024["name"]
        , df_education_attain_2024["Year_2024"]/100.0
    ],
    title="State Educational Attainment in 2024"
)

# To prevent the default hover info which includes the centroid lat-lon coords, we must override the hovertemplate or to cutomize display
# This also requires that we set the hover_data in px.choropleth above not as the hover_data parameter but as the custom_data parameter
# <extra></extra> prevents the lat-lon coords from appearing
bogus = fig_education_attain.update_traces(
    hovertemplate="%{customdata[0]}<br><br>" +
    "%{customdata[1]:.2%}<extra></extra>"
)

In [20]:
df_merged_edu_attain_2024 = pd.merge(df_education_attain_2024, df_state_centroids, on="name", how="inner")
# Create the label text for the map: State Name (Difference)
df_merged_edu_attain_2024["label_text"] = df_merged_edu_attain_2024.apply(
    lambda row: f"{row["name"]}<br>{row["Year_2024"]/100.0:.2%}", axis=1
    # If we only want to show the state name without the associated value(s), then use:
    #lambda row: f"{row["name"]}", axis=1
)
display(df_merged_edu_attain_2024)

,name,Year_2024,lat,lon,leadlat,leadlon,label_text
0,Alaska,32.8,64.2204,-152.5427,64.220400,-152.542700,Alaska<br>32.80%
1,Alabama,29.9,32.7899,-86.8278,32.789900,-86.827800,Alabama<br>29.90%
2,Arkansas,27.1,34.8982,-92.4409,34.898200,-92.440900,Arkansas<br>27.10%
3,Arizona,34.7,34.2934,-111.6633,34.293400,-111.663300,Arizona<br>34.70%
4,California,38.1,37.2539,-119.6144,37.253900,-119.614400,California<br>38.10%
5,Colorado,47.8,38.9993,-105.5488,38.999300,-105.548800,Colorado<br>47.80%
6,Connecticut,42.6,41.6209,-72.7279,40.921887,-71.581369,Connecticut<br>42.60%
7,Delaware,36.0,38.9827,-75.4976,38.679105,-74.487365,Delaware<br>36.00%
8,Florida,35.8,28.6589,-82.5040,28.658900,-82.504000,Florida<br>35.80%
9,Georgia,36.3,32.6480,-83.4465,32.648000,-83.446500,Georgia<br>36.30%


In [21]:
# 6. Add Permanent Text Labels using Centroids
# We use go.Scattergeo to plot the text at the centroid coordinates
bogus = fig_education_attain.add_trace(
    go.Scattergeo(
        locationmode = "USA-states",
        lon = df_merged_edu_attain_2024["leadlon"],
        lat = df_merged_edu_attain_2024["leadlat"],
        text = df_merged_edu_attain_2024["label_text"],
        mode = "text",
        textfont = dict(
            size=12,  # Smaller font to reduce overlap
            color="black"
        ),
        showlegend = False,
        hoverinfo="skip"
    )
)

# Also add leader lines for those smaller states (northeast)

for _, row in df_states_leaderlines.iterrows():
    fig_education_attain.add_trace(go.Scattergeo(
        lon=[row["leadlon"], row["lon"]],
        lat=[row["leadlat"], row["lat"]],
        mode="lines",
        line=dict(color="green", width=1),
        showlegend=False,
        hoverinfo="skip"
    ))

In [22]:
# 7. Final layout adjustments
fig_education_attain.update_geos(
    visible=False,
    # Customize the map projection
    projection={"type": "albers usa"},
    #fitbounds="locations"
)

fig_education_attain.update_layout(
    margin={"r":0,"t":40,"l":0,"b":0},
    coloraxis_colorbar=dict(
        title="Percentage of College Graduates"
    ),
    geo={"scope": "usa"},
    title="Percentage of College Graduates by State"
)

fig_education_attain.show()

In [23]:
fig_education_attain.write_html(f"{output_folder}/PercentageOfCollegeGraduates2024.html")

## Federal Transfers dependency

In [24]:
# For this section of code, we need a function to parse money that might contain commas and dollar signs and parentheses
def parse_money(value_str):
    """
    Convert strings like '$11,892' or '($3,745)' into integers.
    """
    value_str = value_str.strip()

    # Detect negative values wrapped in parentheses
    is_negative = value_str.startswith("(") and value_str.endswith(")")
    value_str = value_str.replace("(", "").replace(")", "")

    # Remove $ and commas
    value_str = value_str.replace("$", "").replace(",", "")

    # Convert to integer
    num = int(value_str)

    return -num if is_negative else num

In [25]:
def parse_state_file(path):
    with open(path, "r", encoding="utf-16") as f:
        lines = [line.strip() for line in f.readlines() if line.strip()]

    # First non-empty line is the state name
    state_name = lines[0]

    data = {"name": state_name}

    # Remaining lines contain "Field<TAB>Value"
    for line in lines[1:]:
        parts = line.split("\t")
        if len(parts) != 2:
            continue  # skip malformed lines

        field, value = parts
        field = field.strip().replace(" ", "")  # remove spaces in keys
        data[field] = parse_money(value)

    return data

In [26]:
all_states = []

glob_csv_files = glob("data/ReceiptsExpendituresTable_*.csv")

for file in glob_csv_files:
    # Read each file and retrieve:
    # From line 1, the state name
    # From each subsquent line, the dollar values, with negative values represented by parentheses, makine sure we strip out commas and dollar signs
    # Each value line contains the field description to the left of the tab character and the dollar value to the right of the tab character
    state_data = parse_state_file(file)
    all_states.append(state_data)

In [27]:
df_fed_transfers = pd.DataFrame(all_states)
display(df_fed_transfers)

,name,DirectPayments,Grants,Contracts,Wages,COVID,IndividualIncomeTax,SocialInsuranceTaxes,CorporateIncomeTaxes,ExciseTaxes,OtherTaxes,Total
0,Alabama,11892,2275,3134,1133,0,-3745,-3767,-860,-275,-59,9728
1,Alaska,8940,5432,6127,4196,0,-5342,-4875,-1205,-239,-50,12983
2,Arizona,10709,3204,2708,687,0,-5187,-4324,-1075,-226,-27,6471
3,Arkansas,10924,3426,443,629,0,-3792,-3572,-941,-279,-100,6738
4,California,9116,3284,1683,750,0,-8278,-5212,-1465,-196,-110,-428
5,Colorado,8819,2310,2669,1342,0,-7457,-5340,-1595,-236,-48,463
6,Connecticut,10056,3024,6977,445,0,-10419,-5852,-1523,-200,-247,2261
7,Delaware,12201,3563,368,783,0,-5346,-4758,-1152,-232,-691,4737
8,Florida,12098,1747,1327,720,0,-8017,-4239,-1329,-209,-120,1978
9,Georgia,9692,2034,1127,1239,0,-5072,-4403,-1012,-217,-42,3345


In [28]:
df_fed_transfers["Expenditures"] = df_fed_transfers["DirectPayments"] + df_fed_transfers["Grants"] + df_fed_transfers["Contracts"] + df_fed_transfers["Wages"] + df_fed_transfers["COVID"]
df_fed_transfers["Receipts"] = -1 * (df_fed_transfers["IndividualIncomeTax"] + df_fed_transfers["SocialInsuranceTaxes"] + df_fed_transfers["CorporateIncomeTaxes"] + df_fed_transfers["ExciseTaxes"] + df_fed_transfers["OtherTaxes"])
df_fed_transfers["Difference"] = df_fed_transfers["Receipts"] - df_fed_transfers["Expenditures"]

final_df_fed_transfers = df_fed_transfers[["name", "Difference", "Expenditures", "Receipts"]]
display(final_df_fed_transfers)

,name,Difference,Expenditures,Receipts
0,Alabama,-9728,18434,8706
1,Alaska,-12984,24695,11711
2,Arizona,-6469,17308,10839
3,Arkansas,-6738,15422,8684
4,California,428,14833,15261
5,Colorado,-464,15140,14676
6,Connecticut,-2261,20502,18241
7,Delaware,-4736,16915,12179
8,Florida,-1978,15892,13914
9,Georgia,-3346,14092,10746


In [29]:
# Define Min, Max, and key transition points
D_min = final_df_fed_transfers["Difference"].min()
D_max = final_df_fed_transfers["Difference"].max()
D_zero = 0
D_violet = -500 # A point for 'negative but near zero'

# Calculate the total range and a helper function for normalization
R = D_max - D_min

def normalize(value, R, D_min):
    """Normalize a value to a fraction between 0 and 1."""
    return (value - D_min) / R

# Define the custom color scale as a list of [normalized_fraction, color_code]
custom_colorscale = [
    # 0.0 (Min: -16097) -> Bright Red (Max negative magnitude)
    [0.0, '#FF0000'],
    # Normalized position of -500 -> Violet (Negative but near zero)
    [normalize(D_violet, R, D_min), '#AA00AA'],
    # Normalized position of 0 -> Light Gray/Neutral (Clear distinction at zero)
    [normalize(D_zero, R, D_min), '#4080FF'],
    # 1.0 (Max: 2758) -> Bright Blue (Max positive value)
    [1.0, '#000080']
]

In [30]:
# 4. Create the Choropleth Map using Plotly Express
fig_fed_transfers = px.choropleth(
    final_df_fed_transfers,
    # Match the 'name' column in the DataFrame with the 'name' property in the GeoJSON features
    geojson=states_geojson,
    locations="name",
    #locationmode="USA-states",
    featureidkey="properties.name",
    
    # Color based on the 'difference' column
    color="Difference",
    
    # Specify the geographic scope (USA)
    scope="usa",
    
    # Set the custom color scale: Red for high positive, Blue for high negative
    # The 'rdbu' (Red-Blue) colorscale is a good diverging option.
    #color_continuous_scale="RdBu",
    color_continuous_scale=custom_colorscale,
    
    #[
    #    [0.0, "rgba(255, 0, 0, 1.0)"],   # red
    #    #[(0 - final_df["Difference"].min()) / (final_df["Difference"].max() - final_df["Difference"].min()), "rgba(128, 0, 128, 1.0)"],   # purple
    #    [1.0, "rgba(0, 0, 255, 1.0)"]    # blue
    #],
    
    # Ensure the color scale centers around zero for accurate representation of positive/negative difference
    #color_continuous_midpoint=0,

    #range_color=(final_df["Difference"].min(), 3*final_df["Difference"].max()),
    #range_color=(final_df["Difference"].min(), final_df["Difference"].max()),
    
    # Add all relevant data to the hover box
    
    #hover_name="name",
    #hover_data={
    #    "Difference": True,
    #    "Expenditures": True,
    #    "Receipts": True,
    #    "name": False, # Hide duplicate name
    #    #'id': False,    # Hide id column
    #    #'metadata': False # Hide original metadata list
    #},
    
    custom_data=[
          final_df_fed_transfers["name"]
        , final_df_fed_transfers["Receipts"]
        , final_df_fed_transfers["Expenditures"]
        , final_df_fed_transfers["Difference"]
    ],
    title="State Difference (Receipts - Expenditures)"
)

# To prevent the default hover info which includes the centroid lat-lon coords, we must override the hovertemplate or to cutomize display
# This also requires that we set the hover_data in px.choropleth above not as the hover_data parameter but as the custom_data parameter
# <extra></extra> prevents the lat-lon coords from appearing
bogus = fig_fed_transfers.update_traces(
    hovertemplate='''
%{customdata[0]}<br><br>
Receipts: $ %{customdata[1]:,}<br>
Expenditures: $ %{customdata[2]:,}<br>
Difference: $ %{customdata[3]:,}
<extra></extra>'''
)

In [31]:
df_merged_fed_transfers = pd.merge(final_df_fed_transfers, df_state_centroids, on="name", how="inner")
# Create the label text for the map: State Name (Difference)
df_merged_fed_transfers["label_text"] = df_merged_fed_transfers.apply(
    lambda row: f"{row["name"]}<br>${row["Difference"]:,d}", axis=1
    # If we only want to show the state name without the associated value(s), then use:
    #lambda row: f"{row["name"]}", axis=1
)
display(df_merged_fed_transfers)

,name,Difference,Expenditures,Receipts,lat,lon,leadlat,leadlon,label_text
0,Alabama,-9728,18434,8706,32.7899,-86.8278,32.789900,-86.827800,"Alabama<br>$-9,728"
1,Alaska,-12984,24695,11711,64.2204,-152.5427,64.220400,-152.542700,"Alaska<br>$-12,984"
2,Arizona,-6469,17308,10839,34.2934,-111.6633,34.293400,-111.663300,"Arizona<br>$-6,469"
3,Arkansas,-6738,15422,8684,34.8982,-92.4409,34.898200,-92.440900,"Arkansas<br>$-6,738"
4,California,428,14833,15261,37.2539,-119.6144,37.253900,-119.614400,California<br>$428
5,Colorado,-464,15140,14676,38.9993,-105.5488,38.999300,-105.548800,Colorado<br>$-464
6,Connecticut,-2261,20502,18241,41.6209,-72.7279,40.921887,-71.581369,"Connecticut<br>$-2,261"
7,Delaware,-4736,16915,12179,38.9827,-75.4976,38.679105,-74.487365,"Delaware<br>$-4,736"
8,Florida,-1978,15892,13914,28.6589,-82.5040,28.658900,-82.504000,"Florida<br>$-1,978"
9,Georgia,-3346,14092,10746,32.6480,-83.4465,32.648000,-83.446500,"Georgia<br>$-3,346"


In [32]:
# 6. Add Permanent Text Labels using Centroids
# We use go.Scattergeo to plot the text at the centroid coordinates
bogus = fig_fed_transfers.add_trace(
    go.Scattergeo(
        locationmode = "USA-states",
        lon = df_merged_fed_transfers["leadlon"],
        lat = df_merged_fed_transfers["leadlat"],
        text = df_merged_fed_transfers["label_text"],
        mode = "text",
        textfont = dict(
            size=12,  # Smaller font to reduce overlap
            color="black",
            weight="bold"
        ),
        showlegend = False,
        hoverinfo="skip"
    )
)

# Also add leader lines for those smaller states (northeast)

for _, row in df_states_leaderlines.iterrows():
    fig_fed_transfers.add_trace(go.Scattergeo(
        lon=[row["leadlon"], row["lon"]],
        lat=[row["leadlat"], row["lat"]],
        mode="lines",
        line=dict(color="green", width=1),
        showlegend=False,
        hoverinfo="skip"
    ))

In [33]:
# 7. Final layout adjustments
fig_fed_transfers.update_geos(
    visible=False,
    # Customize the map projection
    projection={"type": "albers usa"},
    #fitbounds="locations"
)

fig_fed_transfers.update_layout(
    margin={"r":0,"t":40,"l":0,"b":0},
    coloraxis_colorbar=dict(
        title="Difference (Receipts - Expenditures)",
        #tickvals=[D_min, D_violet, D_zero, D_max],
        tickvals=[D_min, D_zero, D_max],
        ticktext=[
            f"Max Negative ({D_min:,}) - Red",
            #f"Near Zero Negative ({D_violet:,}) - Violet",
            "Zero (0) - Neutral",
            f"Max Positive ({D_max:,}) - Blue"
        ]
    ),
    geo={"scope": "usa"},
    title="Difference (Receipts - Expenditures) by State"
)

fig_fed_transfers.show()

In [34]:
fig_fed_transfers.write_html(f"{output_folder}/Federal_Receipts_vs_Expenditures_by_State.html")